In [1]:
# import time
# import requests

# API_BASE = "https://fair2-dev.lutechdigitale.it/med-gemma"

# def predict_image(image_path: str, prompt: str = "<start_of_image> findings:", 
#                   poll_interval: float = 5.0, timeout: float = 600.0) -> str:
#     """
#     1) POST /predict_image → ottieni job_id
#     2) Poll GET /jobs/{job_id} finché status != pending
#     3) Ritorna il result o solleva se status == error
#     """
#     # 1) Submit job
#     with open(image_path, "rb") as f:
#         files = {"file": (image_path, f, "application/octet-stream")}
#         data = {"prompt": prompt}
#         resp = requests.post(f"{API_BASE}/predict_image", files=files, data=data, timeout=(10, 30))
#     resp.raise_for_status()
#     if resp.status_code != 202:
#         raise RuntimeError(f"Unexpected status code {resp.status_code}: {resp.text}")

#     job_id = resp.json().get("job_id")
#     if not job_id:
#         raise RuntimeError(f"No job_id in response: {resp.text}")

#     # 2) Polling loop
#     start = time.time()
#     while True:
#         # Evitiamo di eccedere il timeout complessivo
#         elapsed = time.time() - start
#         if elapsed > timeout:
#             raise TimeoutError(f"Job {job_id} did not complete within {timeout}s")

#         # Richiesta di stato
#         r = requests.get(f"{API_BASE}/jobs/{job_id}", timeout=(5, 30))
#         r.raise_for_status()
#         job = r.json()

#         status = job.get("status")
#         if status == "pending":
#             # ancora in corso: attendi e riprova
#             time.sleep(poll_interval)
#             continue
#         elif status == "done":
#             # tutto ok: restituisci result
#             result = job.get("result")
#             if result is None:
#                 raise RuntimeError(f"Job {job_id} completed without result")
#             return result
#         elif status == "error":
#             # job fallito: dettaglio
#             detail = job.get("detail", "No detail provided")
#             raise RuntimeError(f"Job {job_id} error: {detail}")
#         else:
#             # stato inatteso
#             raise RuntimeError(f"Unknown job status '{status}' for job {job_id}")




In [ ]:
# %% [markdown]
# # Test Prompt Engineering per Analisi Brain Scan - Alzheimer
# 
# Questo notebook testa diversi prompt ottimizzati per l'analisi di immagini cerebrali
# con overlay Grad-CAM usando il modello MedGemma.

# %% [markdown]
## Setup e Import

# %%
import sys
from pathlib import Path
import pandas as pd
import json
from datetime import datetime
import os
import traceback

# Assicuriamoci che la cartella del notebook sia nel path di import
sys.path.append(str(Path().resolve()))

# Import delle funzioni dal modulo models
try:
    from models import predict_image, init_models
    print("✅ Modulo models importato correttamente")
    
    # IMPORTANTE: Inizializza i modelli prima di usarli
    print("🔧 Inizializzando i modelli...")
    try:
        init_models()
        print("✅ Modelli inizializzati correttamente!")
    except Exception as e:
        print(f"❌ Errore durante l'inizializzazione dei modelli: {e}")
        print("Traceback completo:")
        traceback.print_exc()
        
except ImportError as e:
    print(f"❌ Errore nell'importazione del modulo models: {e}")
    print("Assicurati che il file models.py sia nella stessa directory del notebook")
    print("Traceback completo:")
    traceback.print_exc()

# %% [markdown]
## Configurazione Paths e Controlli

# %%
# Cartella immagini organizzata per asse X/Y/Z
BASE_DIR = Path("imagesPrompt")

# Verifica esistenza cartelle
print("📁 Controllo struttura cartelle:")
for axis in ["X", "Y", "Z"]:
    axis_dir = BASE_DIR / axis
    if axis_dir.is_dir():
        img_count = len(list(axis_dir.glob("*.png"))) + len(list(axis_dir.glob("*.jpg"))) + len(list(axis_dir.glob("*.jpeg")))
        print(f"   {axis}/: ✅ {img_count} immagini trovate")
        
        # Elenca i file per debug
        image_files = list(axis_dir.glob("*.png")) + list(axis_dir.glob("*.jpg")) + list(axis_dir.glob("*.jpeg"))
        for img in image_files:
            print(f"      - {img.name} ({img.stat().st_size} bytes)")
    else:
        print(f"   {axis}/: ❌ Directory non trovata")

# %% [markdown]
## Definizione Prompt Ottimizzati

# %%
# Prompt base migliorato
base_prompt = """
TASK: Analyze this Grad-CAM brain visualization for Alzheimer's disease indicators.

CONTEXT:
- This image shows a Grad-CAM heatmap overlaid on a brain scan
- Red/yellow areas indicate high neural network activation (high confidence regions)
- Green/blue areas indicate lower activation
- Focus on medically relevant brain regions for Alzheimer's pathology

ANALYSIS FRAMEWORK:
1. **Highlighted Regions**: Identify specific brain areas with high Grad-CAM activation
2. **Medical Relevance**: Assess if highlighted regions align with known Alzheimer's patterns:
   - Hippocampus (memory formation)
   - Entorhinal cortex (memory gateway)
   - Temporal cortex (semantic memory)
   - Parietal cortex (spatial processing)
3. **Activation Pattern**: Evaluate the distribution and intensity of highlights
4. **Clinical Assessment**: Provide diagnostic confidence and reasoning

REQUIRED OUTPUT FORMAT:
- **Primary Findings**: [Specific observations about highlighted regions]
- **Alzheimer's Indicators**: [Present/Absent with medical justification]
- **Confidence Level**: [High/Medium/Low with reasoning]
- **Recommended Actions**: [Clinical next steps or additional tests needed]

IMPORTANT: Base analysis on established Alzheimer's neuroanatomy and the specific Grad-CAM activation patterns visible in this image.
"""

# Prompt specifici per ogni asse
axis_prompts = {
    "X": """
SPECIALIZED ANALYSIS: Sagittal Brain Slice for Alzheimer's Detection

FOCUS AREAS for this lateral brain view:
- Medial temporal lobe structures
- Posterior cingulate cortex
- Precuneus region
- Corpus callosum integrity

GRAD-CAM INTERPRETATION:
- Examine medial temporal activation patterns
- Assess posterior brain region highlights
- Evaluate activation consistency with Alzheimer's progression

OUTPUT: Provide sagittal-specific anatomical assessment.
""",
    "Y": """
SPECIALIZED ANALYSIS: Coronal Brain Slice for Alzheimer's Detection

FOCUS AREAS for this frontal brain view:
- Hippocampal head and body visualization
- Amygdala region
- Temporal horn enlargement
- Cortical thickness assessment

GRAD-CAM INTERPRETATION:
- Focus on bilateral hippocampal activation
- Assess temporal lobe symmetry in highlights
- Evaluate activation intensity relative to normal patterns

OUTPUT: Provide coronal-specific structural findings.
""",
    "Z": """
SPECIALIZED ANALYSIS: Axial Brain Slice for Alzheimer's Detection

FOCUS AREAS for this horizontal brain slice:
- Hippocampal formations (bilateral)
- Ventricular enlargement (compensatory to atrophy)
- Cortical thickness in temporal regions
- White matter integrity

GRAD-CAM INTERPRETATION:
- Assess if red/yellow highlights align with hippocampal regions
- Evaluate symmetry of activation patterns
- Note any unusual activation in non-typical regions

OUTPUT: Provide axial-specific findings with anatomical precision.
"""
}

# Prompt alternativi per confronto
alternative_prompts = {
    "concise": """
Analyze this Grad-CAM brain scan for Alzheimer's indicators.

Key points:
- Red/yellow = high AI confidence areas
- Focus on hippocampus, temporal cortex, parietal regions
- Assess if activation patterns match known Alzheimer's pathology

Provide:
1. Main findings
2. Alzheimer's likelihood (High/Medium/Low)
3. Key anatomical observations
4. Clinical recommendation

Be specific and medical-accurate.
""",
    
    "detailed": """
COMPREHENSIVE GRAD-CAM BRAIN ANALYSIS FOR ALZHEIMER'S DISEASE

INSTRUCTIONS:
You are analyzing a Grad-CAM visualization overlaid on a brain scan. This technique highlights regions where a deep learning model shows highest confidence in its Alzheimer's vs. normal brain classification.

INTERPRETATION KEY:
- Red/Hot colors: Maximum model activation (highest confidence regions)
- Yellow/Orange: High activation
- Green: Moderate activation  
- Blue/Cold colors: Low activation
- Consider both the presence AND absence of activation in key regions

SYSTEMATIC ANALYSIS REQUIRED:

1. ANATOMICAL REGION IDENTIFICATION:
   - Identify all brain regions showing significant activation
   - Note the intensity and distribution of highlights
   - Compare bilateral symmetry

2. ALZHEIMER'S PATHOLOGY CORRELATION:
   - Hippocampus: Early and severe atrophy in AD
   - Entorhinal cortex: First area affected in AD
   - Temporal cortex: Language and semantic memory
   - Parietal cortex: Spatial processing and attention
   - Posterior cingulate: Default mode network disruption

3. CLINICAL INTERPRETATION:
   - Does the activation pattern support AD diagnosis?
   - Are there unexpected activations that might indicate other conditions?
   - What is the confidence level of this analysis?

4. DIFFERENTIAL CONSIDERATIONS:
   - Could this represent normal aging?
   - Other neurodegenerative conditions?
   - Technical artifacts in the scan?

REQUIRED OUTPUT:
Provide a structured medical report with clear diagnostic reasoning.
"""
}

# %% [markdown]
## Funzione di Test Migliorata

# %%
def test_prompt_on_images(prompt_type="base", limit_per_axis=3, verbose=True, debug=False):
    """
    Testa i prompt sulle immagini organizzate per asse
    
    Args:
        prompt_type: "base", "axis_specific", "concise", "detailed"
        limit_per_axis: numero massimo di immagini per asse da testare
        verbose: se stampare output dettagliato
        debug: se abilitare debug extra
    """
    results = []
    
    # Estensioni immagini supportate
    image_extensions = {".png", ".jpg", ".jpeg", ".bmp", ".tiff"}
    
    # Loop attraverso assi X, Y, Z
    for axis in ["X", "Y", "Z"]:
        axis_dir = BASE_DIR / axis
        
        if not axis_dir.is_dir():
            print(f"⚠️  Directory non trovata: {axis_dir}")
            continue
        
        # Trova tutte le immagini nell'asse
        image_files = []
        for ext in image_extensions:
            image_files.extend(axis_dir.glob(f"*{ext}"))
        
        # Ordina e limita il numero di immagini
        image_files = sorted(image_files)[:limit_per_axis]
        
        if not image_files:
            print(f"⚠️  Nessuna immagine trovata in {axis_dir}")
            continue
        
        print(f"\n🔍 Testando asse {axis} ({len(image_files)} immagini):")
        
        for img_path in image_files:
            try:
                # Verifica che il file esista e sia leggibile
                if not img_path.exists():
                    raise FileNotFoundError(f"File non trovato: {img_path}")
                
                if img_path.stat().st_size == 0:
                    raise ValueError(f"File vuoto: {img_path}")
                
                # Seleziona il prompt appropriato
                if prompt_type == "axis_specific":
                    selected_prompt = f"{base_prompt}\n\n{axis_prompts[axis]}"
                elif prompt_type in alternative_prompts:
                    selected_prompt = alternative_prompts[prompt_type]
                else:
                    selected_prompt = base_prompt
                
                if verbose:
                    print(f"   📷 Analizzando {img_path.name}...", end=" ")
                
                if debug:
                    print(f"\n   📝 Prompt utilizzato: {selected_prompt[:100]}...")
                    print(f"   📁 Path completo: {img_path}")
                
                # Leggi l'immagine come bytes
                with open(img_path, 'rb') as f:
                    image_bytes = f.read()
                
                # Chiamata alla funzione predict_image (che si aspetta bytes, non path)
                result = predict_image(image_bytes, selected_prompt)
                
                if verbose:
                    print("✅")
                    if debug:
                        print(f"      Risultato completo: {result}")
                    else:
                        print(f"      Risultato: {result[:100]}{'...' if len(result) > 100 else ''}")
                
                results.append({
                    "timestamp": datetime.now().isoformat(),
                    "axis": axis,
                    "filename": img_path.name,
                    "prompt_type": prompt_type,
                    "result": result,
                    "status": "success",
                    "result_length": len(result)
                })
                
            except Exception as e:
                error_msg = f"ERROR: {str(e)}"
                if verbose:
                    print(f"❌ {error_msg}")
                
                if debug:
                    print(f"   🔍 Traceback completo:")
                    traceback.print_exc()
                
                results.append({
                    "timestamp": datetime.now().isoformat(),
                    "axis": axis,
                    "filename": img_path.name,
                    "prompt_type": prompt_type,
                    "result": error_msg,
                    "status": "error",
                    "result_length": 0
                })
    
    return results

# %% [markdown]
## Funzione per Testare Inizializzazione Modello

# %%
def test_model_initialization():
    """
    Testa se il modello è correttamente inizializzato
    """
    print("🔧 TESTING MODEL INITIALIZATION")
    print("=" * 40)
    
    try:
        # Verifica che i modelli siano stati inizializzati
        from models import image_model, processor
        
        if image_model is None or processor is None:
            print("❌ Modelli non inizializzati! Chiama init_models() prima del test.")
            return False
            
        print("✅ Variabili globali del modello sembrano OK")
        
        # Trova la prima immagine disponibile per test
        test_img = None
        for axis in ["X", "Y", "Z"]:
            axis_dir = BASE_DIR / axis
            if axis_dir.is_dir():
                imgs = list(axis_dir.glob("*.png")) + list(axis_dir.glob("*.jpg"))
                if imgs:
                    test_img = imgs[0]
                    break
        
        if test_img:
            print(f"🎯 Testando con immagine: {test_img.name}")
            
            # Leggi l'immagine come bytes
            with open(test_img, 'rb') as f:
                image_bytes = f.read()
            
            # Test con prompt semplice
            simple_prompt = "Describe what you see in this image."
            result = predict_image(image_bytes, simple_prompt)
            
            print("✅ Modello funziona correttamente!")
            print(f"   Risultato di test: {result[:100]}{'...' if len(result) > 100 else ''}")
            return True
            
        else:
            print("⚠️ Nessuna immagine trovata per il test")
            return False
            
    except Exception as e:
        print(f"❌ Errore durante il test: {e}")
        traceback.print_exc()
        return False

# %% [markdown]
## Esecuzione Test Preliminari

# %%
# Prima testa l'inizializzazione del modello
model_works = test_model_initialization()

if not model_works:
    print("\n❌ ATTENZIONE: Il modello non è inizializzato correttamente!")
    print("   Controlla il file models.py e assicurati che:")
    print("   1. Il modello sia caricato correttamente")
    print("   2. Le dipendenze siano installate")
    print("   3. I path dei modelli siano corretti")
    print("   4. Le variabili d'ambiente siano configurate")
else:
    print("\n✅ Modello OK, procedo con i test dei prompt")

# %% [markdown]
## Esecuzione Test Prompt (solo se il modello funziona)

# %%
if model_works:
    # Test con prompt base
    print("\n🚀 INIZIO TEST CON PROMPT BASE")
    print("=" * 50)
    
    base_results = test_prompt_on_images(
        prompt_type="base", 
        limit_per_axis=1,  # Inizia con 1 immagine per asse
        verbose=True,
        debug=True  # Abilita debug per il primo test
    )
    
    print(f"\n📊 Test completato: {len(base_results)} immagini processate")
    
    # Se il test base funziona, continua con gli altri
    if any(r['status'] == 'success' for r in base_results):
        print("\n✅ Test base riuscito! Procedo con test aggiuntivi...")
        
        # Test con prompt specifici per asse
        print("\n🚀 TEST CON PROMPT SPECIFICI PER ASSE")
        print("=" * 50)
        
        axis_results = test_prompt_on_images(
            prompt_type="axis_specific", 
            limit_per_axis=1,
            verbose=True
        )
        
        # Test con prompt alternativi
        print("\n🚀 TEST CON PROMPT CONCISO")
        print("=" * 50)
        
        concise_results = test_prompt_on_images(
            prompt_type="concise", 
            limit_per_axis=1,
            verbose=True
        )
        
    else:
        print("\n❌ Test base fallito, controllo necessario prima di continuare")
        axis_results = []
        concise_results = []
        
else:
    print("\n❌ Salto i test perché il modello non funziona")
    base_results = []
    axis_results = []
    concise_results = []

# %% [markdown]
## Analisi e Salvataggio Risultati

# %%
# Combina tutti i risultati
all_results = {
    "base_prompt_results": base_results,
    "axis_specific_results": axis_results,
    "concise_results": concise_results,
    "test_timestamp": datetime.now().isoformat(),
    "model_initialization_test": model_works,
    "test_config": {
        "limit_per_axis": 1,
        "base_dir": str(BASE_DIR),
        "prompt_types_tested": ["base", "axis_specific", "concise"]
    }
}

# Salva in file JSON
results_file = f"test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, indent=2, ensure_ascii=False)

print(f"\n💾 Risultati salvati in: {results_file}")

# Mostra statistiche finali
if base_results:
    df_base = pd.DataFrame(base_results)
    print(f"\n📊 STATISTICHE FINALI:")
    print(f"   - Immagini processate: {len(df_base)}")
    print(f"   - Successi: {len(df_base[df_base['status'] == 'success'])}")
    print(f"   - Errori: {len(df_base[df_base['status'] == 'error'])}")
    
    if any(df_base['status'] == 'success'):
        print(f"   - Lunghezza media risposte: {df_base[df_base['status'] == 'success']['result_length'].mean():.0f} caratteri")

# %% [markdown]
## Troubleshooting e Debug

# %%
print("\n🔍 INFORMAZIONI PER TROUBLESHOOTING:")
print("=" * 50)
print(f"📁 Directory di lavoro: {Path.cwd()}")
print(f"📁 Directory base immagini: {BASE_DIR.resolve()}")
print(f"🐍 Python path: {sys.path}")
print(f"📦 Moduli importati: {[m for m in sys.modules.keys() if 'models' in m]}")

# Controlli aggiuntivi per debugging
print("\n🔍 CONTROLLI AGGIUNTIVI:")
print("=" * 30)

# Controlla se il file models.py esiste
models_file = Path("models.py")
if models_file.exists():
    print(f"✅ File models.py trovato ({models_file.stat().st_size} bytes)")
else:
    print("❌ File models.py NON trovato")

# Controlla le immagini
total_images = 0
for axis in ["X", "Y", "Z"]:
    axis_dir = BASE_DIR / axis
    if axis_dir.is_dir():
        imgs = list(axis_dir.glob("*.png")) + list(axis_dir.glob("*.jpg")) + list(axis_dir.glob("*.jpeg"))
        total_images += len(imgs)

print(f"📷 Totale immagini trovate: {total_images}")

if total_images == 0:
    print("⚠️ ATTENZIONE: Nessuna immagine trovata!")
    print("   Assicurati che le immagini siano nelle cartelle corrette:")
    print("   - imagesPrompt/X/")
    print("   - imagesPrompt/Y/")
    print("   - imagesPrompt/Z/")

✅ Modulo models importato correttamente
🔧 Inizializzando i modelli...
❌ Errore durante l'inizializzazione dei modelli: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/models/medgemma'. Use `repo_type` argument if needed.
Traceback completo:


Traceback (most recent call last):
  File "c:\Users\nicolo.petruzzella\OneDrive - LUTECH SPA\Desktop\promptMRI\medgemma_env\Lib\site-packages\transformers\utils\hub.py", line 470, in cached_files
    hf_hub_download(
    ~~~~~~~~~~~~~~~^
        path_or_repo_id,
        ^^^^^^^^^^^^^^^^
    ...<10 lines>...
        local_files_only=local_files_only,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\nicolo.petruzzella\OneDrive - LUTECH SPA\Desktop\promptMRI\medgemma_env\Lib\site-packages\huggingface_hub\utils\_validators.py", line 106, in _inner_fn
    validate_repo_id(arg_value)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "c:\Users\nicolo.petruzzella\OneDrive - LUTECH SPA\Desktop\promptMRI\medgemma_env\Lib\site-packages\huggingface_hub\utils\_validators.py", line 154, in validate_repo_id
    raise HFValidationError(
    ...<2 lines>...
    )
huggingface_hub.errors.HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/models/medgemma'. U

📁 Controllo struttura cartelle:
   X/: ✅ 1 immagini trovate
      - newplot (1).png (32971 bytes)
   Y/: ✅ 1 immagini trovate
      - newplot (2).png (33700 bytes)
   Z/: ✅ 1 immagini trovate
      - newplot (3).png (55817 bytes)
🔧 TESTING MODEL INITIALIZATION
❌ Modelli non inizializzati! Chiama init_models() prima del test.

❌ ATTENZIONE: Il modello non è inizializzato correttamente!
   Controlla il file models.py e assicurati che:
   1. Il modello sia caricato correttamente
   2. Le dipendenze siano installate
   3. I path dei modelli siano corretti
   4. Le variabili d'ambiente siano configurate

❌ Salto i test perché il modello non funziona

💾 Risultati salvati in: test_results_20250703_184201.json

🔍 INFORMAZIONI PER TROUBLESHOOTING:
📁 Directory di lavoro: c:\Users\nicolo.petruzzella\OneDrive - LUTECH SPA\Desktop\promptMRI
📁 Directory base immagini: C:\Users\nicolo.petruzzella\OneDrive - LUTECH SPA\Desktop\promptMRI\imagesPrompt
🐍 Python path: ['C:\\Users\\nicolo.petruzzella\\AppD

In [ ]:
#img_file = "grad_cam_images_gradcam_visualization116.png"
#prompt = "This image is provided for inference of neural networl classification for alzheimer's disease. Tell me if there is any sign of alzheimer's disease in this image." 
#try:
    #result = predict_image(img_file, prompt)
    #print("Inference result:", result)
#except Exception as e:
    #print("Error during inference:", e)

Inference result: You are a medical AI assistant. Analyze this brain scan and explain what the Grad-CAM highlights.
This image is provided for inference of neural networl classification for alzheimer's disease. Tell me if there is any sign of alzheimer's disease in this image.
Findings:
The Grad-CAM visualization highlights the regions of the brain that are most important for the model's decision-making process. In this case, the visualization shows that the model is focusing on areas such as the hippocampus, amygdala, and cortex. These regions are known to be involved in memory, emotion, and cognitive function, which are all affected in Alzheimer's disease.

Based on the Grad-CAM visualization, it is difficult to definitively diagnose Alzheimer's disease from this single image. However, the model is highlighting areas that are typically affected in Alzheimer's disease, which suggests that the image may contain features that are indicative of the disease. Further analysis of the image,